In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer  # Pour la vectorisation TF-IDF
from sklearn.metrics.pairwise import cosine_similarity       # Pour calculer la similarité cosinus
from sklearn.preprocessing import StandardScaler             # Pour standardiser les caractéristiques numériques
from sklearn.pipeline import FeatureUnion                    # Pour combiner les pipelines de caractéristiques
from sklearn.base import BaseEstimator, TransformerMixin     # Pour créer une classe de transformation personnalisée
from sklearn.preprocessing import OneHotEncoder              # Pour encoder les caractéristiques catégorielles
from nltk.sentiment import SentimentIntensityAnalyzer        # Pour l'analyse des sentiments
import scipy.sparse as sp                                    # Pour gérer les matrices creuses
import numpy as np                                           # Pour les opérations numériques
import pandas as pd                                          # Pour manipuler les données
import nltk                                                  # Pour le traitement du langage naturel

 


Filtrage basé sur le contenu :

Utilise les caractéristiques des articles (ou des utilisateurs) pour trouver des recommandations similaires.
* Avantages : Peut fournir des recommandations explicites et peut surmonter les problèmes de démarrage à froid.
* Inconvénients : Peut manquer de sérendipité et de diversité dans les recommandations.

Chargement des données

In [5]:
df = pd.read_csv('D:\\Projo\\Projet-Data-IA\\Merged_all_tables\\merged_final.csv', sep=',')
df.sort_values(by='averageRating', ascending=False, inplace=True)
df.drop(columns=['poster_path', 'backdrop_path', 'nconst_director'], inplace=True)


1. Téléchargement des ressources nécessaires pour NLTK : 
* Les ressources requises pour la tokenisation, la lemmatisation et les mots vides (stopwords) sont téléchargées à partir de NLTK.

2. Initialisation du stemmer et du lemmatizer : 
* Le stemmer (racinisateur) et le lemmatizer sont initialisés pour réduire les mots à leur base et obtenir les formes de mots canoniques respectivement.

3. Fonction pour appliquer le stemming et la lemmatisation à un texte : 
* Une fonction preprocess_text est définie pour effectuer le prétraitement textuel, y compris la tokenisation, la suppression des mots vides, le stemming et la lemmatisation.

4. Application de la fonction de prétraitement à la colonne "overview" : 
* La fonction preprocess_text est appliquée à la colonne "overview" du DataFrame pour prétraiter le texte des aperçus des films et stocker les résultats dans une nouvelle colonne appelée "processed_overview".

In [6]:
# Initialiser le stemmer et le lemmatizer
stemmer = PorterStemmer()  # Initialiser le stemmer (racinisateur) pour réduire les mots à leur base
lemmatizer = WordNetLemmatizer()  # Initialiser le lemmatizer pour obtenir les formes de mots canoniques

# Fonction pour appliquer le stemming et la lemmatisation à un texte
def preprocess_text(text):
    # Tokenization : Diviser le texte en mots (tokens)
    words = word_tokenize(text)
    # Supprimer les mots vides (stopwords) et mettre en minuscule
    words = [word for word in words if word.lower() not in stopwords.words('english')]
    # Stemming : Réduire les mots à leur racine
    stemmed_words = [stemmer.stem(word) for word in words]
    # Lemmatization : Obtenir les formes canoniques des mots
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
    return ' '.join(lemmatized_words)  # Retourner le texte prétraité sous forme de chaîne de caractères

# Appliquer la fonction de prétraitement à la colonne "overview"
df['processed_overview'] = df['overview'].apply(preprocess_text)


NameError: name 'PorterStemmer' is not defined

1. Importation des bibliothèques : 
* Les bibliothèques nécessaires sont importées pour les différentes opérations de prétraitement des données et de recommandation de films.

2. Définition des colonnes catégorielles et numériques : 
* Les colonnes pertinentes du DataFrame sont spécifiées pour l'analyse et le traitement ultérieurs.

3. Création d'une classe de transformation personnalisée pour extraire les sentiments : 
* Une classe SentimentExtractor est créée pour extraire les sentiments des aperçus de films en utilisant l'analyseur de sentiment NLTK.

4. Création d'un pipeline de transformation pour les caractéristiques : 
* Un pipeline est créé pour combiner la vectorisation TF-IDF des aperçus et l'extraction des sentiments.

5. Application du TfidfVectorizer sur la colonne "processed_overview" : 
* Le TfidfVectorizer est appliqué pour convertir les aperçus en vecteurs TF-IDF.

6. Application du SentimentExtractor sur la colonne "processed_overview" : 
* L'extracteur de sentiments est appliqué pour obtenir les scores de sentiment des aperçus.

7. Concaténation des résultats : 
* Les vecteurs TF-IDF et les scores de sentiment sont concaténés pour obtenir les caractéristiques textuelles combinées.

8. Standardisation des colonnes numériques : 
* Les caractéristiques numériques sont standardisées pour les mettre à la même échelle.

9. Spécification des poids pour chaque colonne numérique : 
* Des poids sont spécifiés pour chaque colonne numérique en fonction de leur importance relative.

10. Application de la pondération aux caractéristiques numériques : 
* Les poids spécifiés sont appliqués aux caractéristiques numériques.

11. Création d'un vecteur de poids pour les caractéristiques catégorielles : 
* Des poids sont spécifiés pour chaque colonne catégorielle en fonction de leur importance relative.

12. Création d'un encodeur one-hot et application de la pondération : 
* Les données catégorielles sont encodées en utilisant un encodeur one-hot et les poids spécifiés sont appliqués.

13. Concaténation des caractéristiques encodées et standardisées : 
* Les caractéristiques textuelles, numériques et catégorielles sont concaténées pour former une seule matrice de caractéristiques.

14. Calcul de la similarité cosinus entre les films : 
* La similarité cosinus est calculée entre les films à l'aide de la matrice de caractéristiques combinée.

15. Fonction de recommandation de films similaires : 
* Une fonction est définie pour recommander des films similaires à un film donné en utilisant la similarité cosinus.

16. Exemple d'utilisation : 
* Un exemple est donné pour obtenir des recommandations pour un film spécifique ("Le Parrain") et les résultats sont affichés.


In [ ]:
# Définir les colonnes catégorielles et numériques
cat_cols = ["title", "genre1", "genre2", "production_companies_name", "Director_name", "Actors_Actresses", "processed_overview"]
num_cols = ["numVotes", "startYear", "runtimeMinutes", "popularity"]

# Créer une classe de transformation personnalisée pour extraire les sentiments de l'aperçu
class SentimentExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.analyzer = SentimentIntensityAnalyzer()  # Initialiser l'analyseur de sentiment

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        sentiments = []
        for overview in X["processed_overview"]:
            sentiment = self.analyzer.polarity_scores(overview)  # Calculer les scores de sentiment
            sentiments.append(list(sentiment.values()))  # Ajouter les scores à la liste
        return sentiments

# Créer un pipeline de transformation pour les caractéristiques
feature_pipeline = FeatureUnion([
    ("tfidf", TfidfVectorizer()),  # Vectorisation TF-IDF pour les aperçus
    ("sentiment", SentimentExtractor())  # Extraction de sentiments pour les aperçus
])

# Appliquer le TfidfVectorizer sur la colonne "processed_overview"
tfidf_vectorizer = TfidfVectorizer()  # Initialiser le vectoriseur TF-IDF
X_tfidf = tfidf_vectorizer.fit_transform(df["processed_overview"])  # Appliquer sur les aperçus

# Appliquer le SentimentExtractor sur la colonne "processed_overview"
sentiment_extractor = SentimentExtractor()  # Initialiser l'extracteur de sentiments
X_sentiment = sentiment_extractor.fit_transform(df)  # Appliquer sur les aperçus

# Concaténer les résultats
X_features = sp.hstack((X_tfidf, X_sentiment), format='csr')  # Concaténer les matrices creuses

# Standardiser les colonnes numériques
scaler = StandardScaler()  # Initialiser le standardiseur
X_numerical = scaler.fit_transform(df[num_cols])  # Standardiser les caractéristiques numériques

# Spécifier les poids pour chaque colonne numérique
num_feature_weights = {
    "numVotes": 1,
    "startYear": 1,
    "runtimeMinutes": 1,
    "popularity": 2,
}

# Appliquer la pondération aux caractéristiques numériques en utilisant les poids spécifiés
X_numerical_weighted = X_numerical.copy()  # Copie des données numériques
for col, weight in num_feature_weights.items():
    col_index = num_cols.index(col)  # Récupérer l'index de la colonne
    X_numerical_weighted[:, col_index] *= weight  # Appliquer la pondération

# Créer un vecteur de poids pour les caractéristiques catégorielles
cat_feature_weights = {
    "title": 2,
    "genre1": 3,
    "genre2": 1,
    "production_companies_name": 2,
    "Director_name": 1,
    "Actors_Actresses": 2,
    "processed_overview": 2
}

# Créer un encodeur one-hot
encoder = OneHotEncoder(handle_unknown='ignore')  # Ignorer les catégories inconnues

# Adapter l'encodeur aux données d'entraînement et le transformer
X_cat_encoded = encoder.fit_transform(df[cat_cols])

# Appliquer la pondération aux caractéristiques catégorielles en utilisant les poids spécifiés
for col, weight in cat_feature_weights.items():
    col_index = cat_cols.index(col)  # Récupérer l'index de la colonne
    X_cat_encoded[:, col_index] *= weight  # Appliquer la pondération

# Concaténer les caractéristiques encodées et standardisées
X = sp.hstack((X_features, X_numerical_weighted, X_cat_encoded), format='csr')  # Concaténer les matrices creuses

# Calculer la similarité cosinus entre les films
cosine_sim = cosine_similarity(X, X)

# Fonction de recommandation de films similaires
def recommend_movies(movie_title, cosine_sim=cosine_sim, df=df):
    idx = df[df['title'] == movie_title].index[0]  # Obtenir l'index du film
    sim_scores = list(enumerate(cosine_sim[idx]))  # Obtenir les scores de similarité
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # Trier par score de similarité
    sim_scores = sim_scores[1:6]  # Exclure le film lui-même
    movie_indices = [i[0] for i in sim_scores]  # Obtenir les indices des films recommandés
    return df['title'].iloc[movie_indices]  # Retourner les titres des films recommandés

# Exemple d'utilisation : obtenir des recommandations pour un film spécifique
movie_title = "Le Roi Lion"
recommended_movies = recommend_movies(movie_title)
print("Recommandations pour", movie_title, ":")
recommended_movies_df


Recommandations pour Le Roi Lion :


0        Forrest Gump
1    Django Unchained
2        Interstellar
3            Parasite
4           Inception
Name: title, dtype: object